# Interpolate Calanus
Interpolate the observations of _Calanus Finmarchicus_ and _Calanus Holandicus_.

The fields are prepared over a region covering the Northeast Atlantic.      
Different resolutions are tested: 0.5° X 0.5°, 1° X 1° and 2° X 2°.

In order to modify the directories, the spatial resolution, edit the [configuration file](./config.ini).

In [1]:
using CSV
using Dates
using DelimitedFiles 
using NCDatasets
using PyPlot
using DIVAnd
using Statistics
const plt=PyPlot
using PyCall
using PyPlot
using TOML
using GeoArrays
colors = PyCall.pyimport("matplotlib.colors")
mticker = PyCall.pyimport("matplotlib.ticker")
cmocean = PyCall.pyimport("cmocean")
plt.rc("font", family="Times New Roman", size=16)
include("../src/InterpCalanus.jl")
makeplot = false
usecartopy = false

false

## User settings
Read configuration file

In [2]:
config = TOML.parsefile("./config.ini")
dom = config["domain"]

4-element Vector{Float64}:
 -20.5
  11.75
  41.25
  67.0

## Install `Cartopy`
Set `usecartopy` to `false` if you don't want to use it.

In [3]:
if usecartopy
    @info("Will try to use Cartopy for the plots")
    cartopy = PyCall.pyimport("cartopy")
    ccrs = PyCall.pyimport("cartopy.crs")
    cfeature = PyCall.pyimport("cartopy.feature")
    mticker = PyCall.pyimport("matplotlib.ticker")
    coast = cfeature.GSHHSFeature(scale="full");
    cartopyticker = PyCall.pyimport("cartopy.mpl.ticker")
    lon_formatter = cartopyticker.LongitudeFormatter()
    lat_formatter = cartopyticker.LatitudeFormatter()
    mainproj = ccrs.Mercator(central_longitude=0.5*(dom[1] + dom[2]), 
        min_latitude=dom[3], max_latitude=dom[4])
    datacrs = ccrs.PlateCarree();
else
    @info("Won't use cartopy")
end

┌ Info: Won't use cartopy
└ @ Main In[3]:15


## User inputs
### Type of analysis to be performed

In [4]:
run_year = true
run_month = true

true

### Files and directories

In [5]:
resdir = config["directories"]["resdir"]
figdir = config["directories"]["figdir"]
isdir("resdir") ? @debug("already there") : mkpath("resdir")
isdir(figdir) ? @debug("already there") : mkpath(figdir)
datadir = config["directories"]["datadir"]
datafile = joinpath(datadir, config["files"]["datafile"])
isfile(datafile) ? @debug("already downloaded") : download(config["files"]["datafileURL"], datafile)
bathfile = joinpath(datadir, config["files"]["bathyfile"])
isfile(datafile), isfile(bathfile)

(true, true)

## Grid configuration

In [6]:
longrid = dom[1]:config["dlon"]:dom[2]
latgrid = dom[3]:config["dlat"]:dom[4]
@info("Size of the grid: $(length(longrid)) X $(length(latgrid))")

┌ Info: Size of the grid: 130 X 104
└ @ Main In[6]:3


### Metrics

In [7]:
_, (pm, pn), (xi, yi) = DIVAnd_rectdom(longrid, latgrid);
@info(size(xi));

┌ Info: (130, 104)
└ @ Main In[7]:2


### Prepare land-sea mask

In [8]:
bx, by, b = DIVAnd.load_bath(bathfile, true, longrid, latgrid)
@show size(b)

size(b) = (130, 104)


(130, 104)

In [9]:
_, _, mask = DIVAnd.load_mask(bathfile, true, longrid, latgrid, 5.0);
# Remove Mediterranean Sea

# - Resolution = 0.1°
# mask[220:end,1:60] .= 0;

# - Resolution = 0.5°
maskindex = findall((xi .>= 0.) .& (yi .<= 47.))
mask[maskindex] .= 0.;

## Data reading
### Read the CSV files and separate by columns

In [10]:
@time lon, lat, dates, calanus_finmarchicus, calanus_helgolandicus = InterpCalanus.read_data_calanus(datafile);

  1.745828 seconds (5.64 M allocations: 190.164 MiB, 3.01% gc time, 66.59% compilation time: 3% of which was recompilation)


┌ Info: (-20.447, 11.703)
└ @ Main.InterpCalanus /home/ctroupin/Projects/EMODnet/EMODnet-Bio-Interp-Calanus/src/InterpCalanus.jl:39
┌ Info: (41.2567, 66.808)
└ @ Main.InterpCalanus /home/ctroupin/Projects/EMODnet/EMODnet-Bio-Interp-Calanus/src/InterpCalanus.jl:40


## Analysis
### 1. Basic analysis: all data together

In [11]:
L = 2.5
epsilon2 = 5.

5.0

In [12]:
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (L, L), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (L, L), epsilon2);

 26.147841 seconds (48.08 M allocations: 2.666 GiB, 4.19% gc time, 97.35% compilation time)
  0.476857 seconds (2.07 M allocations: 295.693 MiB, 4.58% gc time, 9.11% compilation time)


### 2. Long correlation length

In [13]:
Llong = 100
epsilon2 = 5.
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (Llong, Llong), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (Llong, Llong), epsilon2);

if makeplot
    figname = joinpath(figdir, "analysis_calanus_finmarchicus_large.jpg")
    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                       "Calanus finmarchicus analysis (L = $(Llong), eps2 = $(epsilon2))", figname,
                       mainproj)
end

  3.141856 seconds (6.33 M allocations: 506.518 MiB, 15.96% gc time, 67.96% compilation time)
  0.795094 seconds (2.05 M allocations: 294.422 MiB, 35.97% gc time)


### 3. Short correlation length

In [14]:
Lshort = 0.5
epsilon2 = 5.
@time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_finmarchicus .- 0. * mean(calanus_finmarchicus), (Lshort, Lshort), epsilon2);
@time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon, lat), 
    calanus_helgolandicus .- 0. * mean(calanus_helgolandicus), (Lshort, Lshort), epsilon2);

if makeplot
    figname = joinpath(figdir, "analysis_calanus_finmarchicus_short.jpg")
    PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                       "Calanus finmarchicus analysis (L = $(Lshort), eps2 = $(epsilon2))", figname,
                       mainproj)
end

  0.494926 seconds (2.05 M allocations: 294.423 MiB, 3.29% gc time)
  0.527618 seconds (2.05 M allocations: 294.422 MiB, 3.47% gc time)


## Time analysis
### Yearly
In these yearly analysis all the data are taken, whatever the month.     
Create the (empty) netCDF files:

In [15]:
include("../src/InterpCalanus.jl")
resfileyear1 = joinpath(resdir, "calanus_finmarchicus_year.nc")
resfileyear2 = joinpath(resdir, "calanus_helgolandicus_year.nc")
InterpCalanus.create_nc_results_time(resfileyear1, longrid, latgrid,  
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=dom,
                      aphiaID=Int32(104464), L, epsilon2);

InterpCalanus.create_nc_results_time(resfileyear2, longrid, latgrid,  
                      "Calanus helgolandicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus helgolandicus",
                      domain=dom,
                      aphiaID=Int32(104466), L, epsilon2);

Loop on the years:

In [16]:
if run_year 
    for (iii, yyyy) in enumerate(1959:2018)
        goodyear = findall(Dates.year.(dates) .== yyyy)
        @info("Number of observations for year $(yyyy): $(length(goodyear))")

        # Analysis
        f_finmarchicus, s_fin = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_finmarchicus[goodyear] .- 0. * mean(calanus_finmarchicus[goodyear]), (L, L), epsilon2);
        f_helgolandicus, s_hel = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodyear], lat[goodyear]), 
            calanus_helgolandicus[goodyear] .- 0. * mean(calanus_helgolandicus[goodyear]), (L, L), epsilon2);
        
        # Error field
        cpme = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodyear], lat[goodyear]), 
            calanus_finmarchicus[goodyear], L, epsilon2);
        
        #cpme_helgo = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodyear], lat[goodyear]), 
        #    calanus_helgolandicus[goodyear], L, epsilon2);
        
        if makeplot
            # Figures
            figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), 
                "Calanus finmarchicus analysis ($(yyyy))", figname, mainproj)

            figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_analysis(longrid, latgrid, f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), 
                "Calanus helgolandicus analysis ($(yyyy))", figname, mainproj)

            figname = joinpath(figdir, "error_calanus_finmarchicus_$(yyyy).jpg")
            PlottingCalanus.make_plot_error(longrid, latgrid, cpme, lon[goodyear], lat[goodyear], 
                "Relative error field", figname, mainproj)
        end
        
        #figname = joinpath(figdir, "error_calanus_helgolandicus_$(yyyy).jpg")
        #PlottingCalanus.make_plot_error(longrid, latgrid, cpme_helgo, lon[goodyear], lat[goodyear], 
        #    "Relative error field", figname, mainproj)
        
        # write single netCDF files
        InterpCalanus.create_nc_results(joinpath(resdir, "finmarchicus_$(yyyy).nc"), 
                      longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
                      f_finmarchicus .+ 0. * mean(calanus_finmarchicus[goodyear]), L, epsilon2,
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=dom,
                      aphiaID=Int32(104464)
                   )
        
        InterpCalanus.write_nc_error(joinpath(resdir, "finmarchicus_$(yyyy).nc"), cpme)

        InterpCalanus.create_nc_results(joinpath(resdir, "helgolandicus_$(yyyy).nc"), 
                  longrid, latgrid, (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value, 
                  f_helgolandicus .+ 0. * mean(calanus_helgolandicus[goodyear]), L, epsilon2, 
                  "Calanus helgolandicus"; valex=-999.9, 
                  varname="abundance",
                  long_name="Abundance of Calanus helgolandicus",
                  domain=dom,
                  aphiaID=Int32(104466)
               )
        
        InterpCalanus.write_nc_error(joinpath(resdir, "helgolandicus_$(yyyy).nc"), cpme)
        
        # Write common netCDF files
        Dataset(resfileyear1, "a") do ds
            ds["time"][iii] = (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value
            ds["error"][:,:,iii] = cpme
            ds["abundance"][:,:,iii] = f_finmarchicus
        end

        Dataset(resfileyear2, "a") do ds
            ds["time"][iii] = (Dates.Date(yyyy, 1, 1) - Dates.Date(1950, 1, 1)).value
            ds["error"][:,:,iii] = cpme
            ds["abundance"][:,:,iii] = f_helgolandicus
        end
        
        # geoTIFF files
        InterpCalanus.write_geotiff(joinpath(resdir, "finmarchicus_$(yyyy).tif"), f_finmarchicus, dom)
        InterpCalanus.write_geotiff(joinpath(resdir, "helgolandicus_$(yyyy).tif"), f_helgolandicus, dom)
        InterpCalanus.write_geotiff(joinpath(resdir, "finmarchicus_error_$(yyyy).tif"), cpme, dom)
        #InterpCalanus.write_geotiff(joinpath(resdir, "helgolandicus_error_$(yyyy).tif"), cpme_helgo, dom)

    end
end

┌ Info: Number of observations for year 1959: 2364
└ @ Main In[16]:4
┌ Info: Number of observations for year 1960: 2321
└ @ Main In[16]:4
┌ Info: Number of observations for year 1961: 2382
└ @ Main In[16]:4
┌ Info: Number of observations for year 1962: 2564
└ @ Main In[16]:4
┌ Info: Number of observations for year 1963: 2465
└ @ Main In[16]:4
┌ Info: Number of observations for year 1964: 2556
└ @ Main In[16]:4
┌ Info: Number of observations for year 1965: 2777
└ @ Main In[16]:4
┌ Info: Number of observations for year 1966: 2422
└ @ Main In[16]:4
┌ Info: Number of observations for year 1967: 2679
└ @ Main In[16]:4
┌ Info: Number of observations for year 1968: 2489
└ @ Main In[16]:4
┌ Info: Number of observations for year 1969: 2863
└ @ Main In[16]:4
┌ Info: Number of observations for year 1970: 2701
└ @ Main In[16]:4
┌ Info: Number of observations for year 1971: 2964
└ @ Main In[16]:4
┌ Info: Number of observations for year 1972: 2808
└ @ Main In[16]:4
┌ Info: Number of observations for

### Monthly

In [17]:
resfilemonth1 = joinpath(resdir, "calanus_finmarchicus_year_month.nc")
resfilemonth2 = joinpath(resdir, "calanus_helgolandicus_year_month.nc")

InterpCalanus.create_nc_results_time(resfilemonth1, longrid, latgrid,  
                      "Calanus finmarchicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus finmarchicus",
                      domain=dom,
                      aphiaID=Int32(104464), L, epsilon2);

InterpCalanus.create_nc_results_time(resfilemonth2, longrid, latgrid,  
                      "Calanus helgolandicus"; valex=-999.9, 
                      varname="abundance",
                      long_name="Abundance of Calanus helgolandicus",
                      domain=dom,
                      aphiaID=Int32(104466), L, epsilon2);

In [18]:
iii = 0
for yyyy in 1959:2018

    for m in 1:12
        
        iii += 1

        mm = lpad(string(m), 2, '0')
        goodtime = findall( (Dates.year.(dates) .== yyyy) .& (Dates.month.(dates) .== m) )
        @info("Found $(length(goodtime)) dates points for year $(yyyy) | month $(mm)")
        
        if length(goodtime) > 0

            @time f_finmarchicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                calanus_finmarchicus[goodtime] .- 0. * mean(calanus_finmarchicus[goodtime]), (L, L), epsilon2);
            @time f_helgolandicus, s = DIVAndrun(mask, (pm,pn), (xi,yi), (lon[goodtime], lat[goodtime]), 
                calanus_helgolandicus[goodtime] .- 0. * mean(calanus_helgolandicus[goodtime]), (L, L), epsilon2);
            
            # Error field
            cpme_fin = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                calanus_finmarchicus[goodtime], L, epsilon2);
            cpme_hel = DIVAnd_cpme(mask, (pm, pn), (xi, yi), (lon[goodtime], lat[goodtime]), 
                calanus_helgolandicus[goodtime], L, epsilon2);
            
            if makeplot
                figname = joinpath(figdir, "analysis_calanus_finmarchicus_$(yyyy)$(mm).jpg")         
                PlottingCalanus.make_plot_analysis(longrid, latgrid, f_finmarchicus, 
                "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname, mainproj)

                figname = joinpath(figdir, "analysis_calanus_helgolandicus_$(yyyy)$(mm).jpg")
                #make_plot_analysis(longrid, latgrid, f_helgolandicus .+ mean(calanus_helgolandicus[goodtime]), 
                #    "Calanus finmarchicus analysis ($(Dates.monthname(m)) $(yyyy))", figname)
            end
         
            Dataset(resfilemonth1, "a") do ds
                ds["time"][iii] = (Dates.Date(yyyy, m, 1) - Dates.Date(1950, 1, 1)).value
                ds["error"][:,:,iii] = cpme_fin
                ds["abundance"][:,:,iii] = f_finmarchicus
            end
            
            Dataset(resfilemonth2, "a") do ds
                ds["time"][iii] = (Dates.Date(yyyy, m, 1) - Dates.Date(1950, 1, 1)).value
                ds["error"][:,:,iii] = cpme_hel
                ds["abundance"][:,:,iii] = f_helgolandicus
            end
            
        else
            @warn("No data to perform analysis")
        end

    end

end

┌ Info: Found 141 dates points for year 1959 | month 01
└ @ Main In[18]:10


  0.093676 seconds (13.89 k allocations: 118.897 MiB)
  0.109175 seconds (4.59 k allocations: 118.374 MiB, 14.97% gc time)
  0.080792 seconds (4.62 k allocations: 118.377 MiB)


┌ Info: Found 144 dates points for year 1959 | month 02
└ @ Main In[18]:10


  0.086654 seconds (4.62 k allocations: 118.378 MiB, 13.73% gc time)
  0.079631 seconds (5.97 k allocations: 118.525 MiB, 8.51% gc time)


┌ Info: Found 247 dates points for year 1959 | month 03
└ @ Main In[18]:10


...

  0.106707 seconds (5.62 k allocations: 118.491 MiB)
  0.085482 seconds (5.62 k allocations: 118.491 MiB, 4.00% gc time)
  0.081342 seconds (5.28 k allocations: 118.448 MiB, 4.00% gc time)


┌ Info: Found 194 dates points for year 2018 | month 11
└ @ Main In[18]:10


  0.068675 seconds (5.26 k allocations: 118.448 MiB)
  0.072963 seconds (5.51 k allocations: 118.477 MiB)


┌ Info: Found 213 dates points for year 2018 | month 12
└ @ Main In[18]:10


  0.077849 seconds (5.52 k allocations: 118.477 MiB, 3.95% gc time)


In [19]:
@info("Results writtin in directory $(resdir)");

┌ Info: Results writtin in directory ../results/L025/
└ @ Main In[19]:1
